https://projecteuler.net/index.php?section=problems&id=30

https://discourse.julialang.org/t/reduce-number-of-allocations/66990

In [1]:
using BenchmarkTools

In [2]:
function power_digit_sum(pow, n)
    return sum(c^pow for c in reverse(digits(n)))
end

function Problem30()
    ans = sum(i for i in 2:1_000_000 if i == power_digit_sum(5, i))
    return ans
end

@show Problem30();
@btime Problem30();

Problem30() = 443839
  130.321 ms (2000004 allocations: 243.83 MiB)


https://docs.julialang.org/en/v1/base/numbers/#Base.digits!

In [3]:
module Rev1

function power_digit_sum!(pow, n, ws)
    return sum(c^pow for c in digits!(ws, n))
end

function Problem30(N, m)
    ws = Vector{Int}(undef, N)
    ans = sum(i for i in 2:(10^N - 1) if i == power_digit_sum!(m, i, ws))
    return ans
end

end

@show Rev1.Problem30(5, 4);
@show Rev1.Problem30(6, 5);
@btime Rev1.Problem30(6, 5);

Rev1.Problem30(5, 4) = 19316
Rev1.Problem30(6, 5) = 443839
  50.873 ms (1 allocation: 128 bytes)


In [4]:
module DNF1

function powsum(pow, n)
    s = 0
    while n > 0
        (n, r) = divrem(n, 10)
        s += r^pow
    end
    return s
end

problem30() = sum(i for i in 2:1_000_000 if i == powsum(5, i))

end

@show DNF1.problem30();
@btime DNF1.problem30();

DNF1.problem30() = 443839
  22.458 ms (0 allocations: 0 bytes)


In [5]:
module DNF2

function pow5(x)
    y = x^2
    return y^2 * x
end

function pow5sum(n)
    s = 0
    while n >= 10
        (n, r) = divrem(n, 10)
        s += pow5(r)
    end
    return s + pow5(n)
end

problem30() = sum(i for i in 2:1_000_000 if i == pow5sum(i))

end

@show DNF2.problem30();
@btime DNF2.problem30();

DNF2.problem30() = 443839
  8.376 ms (0 allocations: 0 bytes)


https://docs.julialang.org/en/v1/devdocs/cartesian/

In [6]:
using Base.Cartesian

In [7]:
(@macroexpand @nexprs 4 d -> s += 10^(d-1)*i_d) |> Base.remove_linenums!

quote
    s += 1i_1
    s += 10i_2
    s += 100i_3
    s += 1000i_4
end

In [8]:
(@macroexpand @nloops 4 i d -> 0:9 begin
    @nexprs 4 d -> s += 10^(d-1)*i_d
end) |> Base.remove_linenums!

quote
    for i_4 = 0:9
        nothing
        begin
            for i_3 = 0:9
                nothing
                begin
                    for i_2 = 0:9
                        nothing
                        begin
                            for i_1 = 0:9
                                nothing
                                begin
                                    begin
                                        s += 1i_1
                                        s += 10i_2
                                        s += 100i_3
                                        s += 1000i_4
                                    end
                                end
                                nothing
                            end
                        end
                        nothing
                    end
                end
                nothing
            end
        end
        nothing
    end
end

In [9]:
N = 4
s = 0
@eval @nloops $N i d -> 0:9 begin
    @nexprs $N d -> global s += 10^(d-1)*i_d
end
s

49995000

In [10]:
sum(0:9999)

49995000

In [11]:
begin
    begin
        a = Int[]
        s = 0
        x = 0
        @nloops 5 i (d -> 0:9) begin
            y = 0
            @nexprs 5 d -> y += i_d ^ 4
            if x > 1 && x == y
                push!(a, x)
                s += x
            end
            x += 1
        end
        a, s
    end
end

([1634, 8208, 9474], 19316)

In [12]:
(@macroexpand begin
    begin
        p = @ntuple 10 i -> (i - 1) ^ 5
        a = Int[]
        s = 0
        x = 0
        @inbounds @nloops 6 i d -> 0:9 d -> y_d = p[i_d+1] begin
            y = 0
            @nexprs 6 d -> y += y_d
            if x > 1 && x == y
                push!(a, x)
                s += x
            end
            x += 1
        end
        a, s
    end
end) |> Base.remove_linenums!

quote
    begin
        p = (0, 1, 32, 243, 1024, 3125, 7776, 16807, 32768, 59049)
        a = Int[]
        s = 0
        x = 0
        begin
            $(Expr(:inbounds, true))
            local var"#38#val" = begin
                        for i_6 = 0:9
                            y_6 = p[i_6 + 1]
                            begin
                                for i_5 = 0:9
                                    y_5 = p[i_5 + 1]
                                    begin
                                        for i_4 = 0:9
                                            y_4 = p[i_4 + 1]
                                            begin
                                                for i_3 = 0:9
                                                    y_3 = p[i_3 + 1]
                                                    begin
                                                        for i_2 = 0:9
                                                            y_2 = p[i_2 + 1]
                      

In [13]:
@generated function _projecteuler30v(::Val{N}, ::Val{m}) where {N, m}
    quote
        p = @ntuple 10 i -> (i - 1) ^ $m
        a = Int[]
        s = 0
        x = 0
        @inbounds @nloops $N i d -> 0:9 d -> y_d = p[i_d+1] begin
            y = 0
            @nexprs $N d -> y += y_d
            if x > 1 && x == y
                push!(a, x)
                s += x
            end
            x += 1
        end
        a, s
    end
end
projecteuler30v(N, m) = _projecteuler30v(Val(N), Val(m))

@show projecteuler30v(5, 4);
@show projecteuler30v(6, 5);
@btime projecteuler30v(6, 5);

projecteuler30v(5, 4) = ([1634, 8208, 9474], 19316)
projecteuler30v(6, 5) = ([4150, 4151, 54748, 92727, 93084, 194979], 443839)
  840.700 μs (3 allocations: 208 bytes)


In [14]:
@generated function _projecteuler30(::Val{N}, ::Val{m}) where {N, m}
    quote
        p = @ntuple 10 i -> (i - 1) ^ $m
        s = 0
        x = 0
        @inbounds @nloops $N i d -> 0:9 begin
            y = 0
            @nexprs $N d -> y += p[i_d+1]
            s += (x == y) * x
            x += 1
        end
        s - 1
    end
end
projecteuler30(N, m) = _projecteuler30(Val(N), Val(m))

@show projecteuler30(5, 4);
@show projecteuler30(6, 5);
@btime projecteuler30(6, 5);

projecteuler30(5, 4) = 19316
projecteuler30(6, 5) = 443839
  298.000 μs (0 allocations: 0 bytes)


In [15]:
@generated function _projecteuler30x(::Val{N}, ::Val{m}) where {N, m}
    quote
        p = @ntuple 10 i -> (i - 1) ^ $m
        s = 0
        x = 0
        @inbounds @nloops $N i d -> 0:9 d -> begin
            y_d = p[i_d+1]
        end begin
            y = 0
            @nexprs $N d -> y += y_d
            s += (x == y) * x
            x += 1
        end
        s - 1
    end
end
projecteuler30x(N, m) = _projecteuler30x(Val(N), Val(m))

@show projecteuler30x(5, 4);
@show projecteuler30x(6, 5);
@btime projecteuler30x(6, 5);

projecteuler30x(5, 4) = 19316
projecteuler30x(6, 5) = 443839
  297.900 μs (0 allocations: 0 bytes)


In [16]:
function projecteuler30p()
    p = @ntuple 10 i -> (i - 1) ^ 5
    s = 0
    x = 0
    @inbounds @nloops 6 i d -> 0:9 begin
        y = 0
        @nexprs 6 d -> y += p[i_d+1]
        s += (x == y) * x
        x += 1
    end
    s - 1
end

@show projecteuler30p();
@btime projecteuler30p();

projecteuler30p() = 443839
  297.900 μs (0 allocations: 0 bytes)


In [17]:
function projecteuler30()
    s = 0
    x = 0
    @nloops 6 i d -> 0:9 begin
        y = 0
        @nexprs 6 d -> y += (z = i_d^2; z^2*i_d)
        s += (x == y) * x
        x += 1
    end
    s - 1
end

@show projecteuler30();
@btime projecteuler30();

projecteuler30() = 443839
  237.700 μs (0 allocations: 0 bytes)


In [18]:
function projecteuler30x()
    s = 0
    x = 0
    @nloops 6 i d -> 0:9 d -> begin
        z = i_d^2
        y_d = z^2 * i_d
    end begin
        y = 0
        @nexprs 6 d -> y += y_d
        s += (x == y) * x
        x += 1
    end
    s - 1
end

@show projecteuler30x();
@btime projecteuler30x();

projecteuler30x() = 443839
  237.700 μs (0 allocations: 0 bytes)


In [19]:
[(k, k * 9^4, log10(k * 9^4)+1) for k in 1:9]

9-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 6561, 4.816970037757299)
 (2, 13122, 5.11800003342128)
 (3, 19683, 5.294091292476962)
 (4, 26244, 5.4190300290852615)
 (5, 32805, 5.515940042093319)
 (6, 39366, 5.595121288140943)
 (7, 45927, 5.662068077771556)
 (8, 52488, 5.720060024749243)
 (9, 59049, 5.771212547196624)

In [20]:
[(k, k * 9^5, log10(k * 9^5)+1) for k in 1:9]

9-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 59049, 5.771212547196624)
 (2, 118098, 6.0722425428606055)
 (3, 177147, 6.248333801916287)
 (4, 236196, 6.373272538524587)
 (5, 295245, 6.470182551532643)
 (6, 354294, 6.549363797580268)
 (7, 413343, 6.616310587210881)
 (8, 472392, 6.674302534188568)
 (9, 531441, 6.7254550566359494)

In [21]:
[(k, k * 9^10, log10(k * 9^10)+1) for k in 1:12]

12-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 3486784401, 10.542425094393248)
 (2, 6973568802, 10.84345509005723)
 (3, 10460353203, 11.019546349112911)
 (4, 13947137604, 11.144485085721211)
 (5, 17433922005, 11.241395098729267)
 (6, 20920706406, 11.320576344776892)
 (7, 24407490807, 11.387523134407505)
 (8, 27894275208, 11.445515081385192)
 (9, 31381059609, 11.496667603832574)
 (10, 34867844010, 11.542425094393248)
 (11, 38354628411, 11.583817779551474)
 (12, 41841412812, 11.621606340440874)

In [22]:
[(m, findfirst(k -> k > log10(k*10.0^m)+1, 1:2m)-1) for m in 2:100]

99-element Vector{Tuple{Int64, Int64}}:
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 6)
 (6, 7)
 (7, 8)
 (8, 10)
 (9, 11)
 (10, 12)
 (11, 13)
 (12, 14)
 (13, 15)
 (14, 16)
 ⋮
 (89, 91)
 (90, 92)
 (91, 93)
 (92, 94)
 (93, 95)
 (94, 96)
 (95, 97)
 (96, 98)
 (97, 100)
 (98, 101)
 (99, 102)
 (100, 103)

In [23]:
?@nloops

```
@nloops N itersym rangeexpr bodyexpr
@nloops N itersym rangeexpr preexpr bodyexpr
@nloops N itersym rangeexpr preexpr postexpr bodyexpr
```

Generate `N` nested loops, using `itersym` as the prefix for the iteration variables. `rangeexpr` may be an anonymous-function expression, or a simple symbol `var` in which case the range is `axes(var, d)` for dimension `d`.

Optionally, you can provide "pre" and "post" expressions. These get executed first and last, respectively, in the body of each loop. For example:

```
@nloops 2 i A d -> j_d = min(i_d, 5) begin
    s += @nref 2 A j
end
```

would generate:

```
for i_2 = axes(A, 2)
    j_2 = min(i_2, 5)
    for i_1 = axes(A, 1)
        j_1 = min(i_1, 5)
        s += A[j_1, j_2]
    end
end
```

If you want just a post-expression, supply [`nothing`](@ref) for the pre-expression. Using parentheses and semicolons, you can supply multi-statement expressions.


In [24]:
(@macroexpand @nloops 3 i d -> 0:9 d -> y_d = i_d^2 begin
    y = 0
    @nexprs 3 d -> y += y_d
end)|> Base.remove_linenums!

quote
    for i_3 = 0:9
        y_3 = i_3 ^ 2
        begin
            for i_2 = 0:9
                y_2 = i_2 ^ 2
                begin
                    for i_1 = 0:9
                        y_1 = i_1 ^ 2
                        begin
                            y = 0
                            begin
                                y += y_1
                                y += y_2
                                y += y_3
                            end
                        end
                        nothing
                    end
                end
                nothing
            end
        end
        nothing
    end
end